In [ ]:
spark

In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
from io import BytesIO
import re
# import tensorflow as tf
# import cv2

I will use the functionality of Image Datasource of spark to collect and process images in bytes, and then start processing the images for the neural network

In [ ]:
images_folders = ['/CA1/Images/Ireland/','/CA1/Images/Honduran/']

In [ ]:
images_rdd = spark.sparkContext.binaryFiles(','.join(images_folders))
# images_rdd = spark.sparkContext.binaryFiles('hdfs://172.24.144.178:9000/CA1/Images/Ireland/Ireland_001.jpeg')

In [ ]:
images_rdd

In [ ]:
def extract_data(data):
    file_path, image_data = data
#     image = Image.open(BytesIO(image_data))
#     image_array = np.array(image)
    
    file_name = file_path.split('/')[-1]
    file_name_without_ext = file_name.split(".")[0]
    label,name = file_name_without_ext.split('_')[0], file_name_without_ext.split('_')[1]
    
    return name, label, image_data

In [ ]:
imageDf = images_rdd.map(lambda x: extract_data(x)).toDF(["Name","Label","Data"])

In [ ]:
pandasImagesDF = imageDf.toPandas()

In [ ]:
def processImage(data, target_size=(225,225)):
    imgbytes = BytesIO(data)
    image = Image.open(imgbytes)
    resized_img = image.resize(target_size, Image.ANTIALIAS)
    with BytesIO() as output:
        resized_img.save(output, format="PNG")
        new_image = Image.open(output)

        array = np.asarray(new_image).reshape([target_size[0],target_size[1],3])
        return array

In [ ]:
pandasImagesDF["Data"] = pandasImagesDF["Data"].apply(lambda x: processImage(x))

In [ ]:
# pandasImagesDF

In [ ]:
pandasImagesDF = pandasImagesDF.sort_values(by=["Name"])

In [ ]:
pandasImagesDF.info()

In [ ]:
#import matplotlib.pyplot as plt

In [ ]:
#np.array(pandasImagesDF["Data"][9]).astype("float32").shape

In [ ]:
# # Determine the dimensions of the image
# width =  225
# height =  225

# # Convert the byte array to a numpy array of uint8 data type
# image_data = np.array(pandasImagesDF["Data"][0], dtype=np.uint8)
# # image_data.reshape(-1,1)
# # Reshape the array to match the dimensions of the image
# # image_data = image_data.reshape((height, width))

# # Create an image from the byte array
# image = Image.fromarray(image_data)

# # Display the image
# image.show()

Importing from Keras functionality necessary to implement CNN

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from tensorflow.keras import utils
from sklearn.metrics import accuracy_score

For Now there is only 20 photos per Category, the training size will be 28 and test 12, This is for testing. After this I'll get more images to improve the CNN

In [ ]:
X_train = pandasImagesDF.iloc[:-12,2].values
y_train = pandasImagesDF.iloc[:-12,1].values
X_test = pandasImagesDF.iloc[-12:,2].values
y_test = pandasImagesDF.iloc[-12:,1].values


In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
encoder = OneHotEncoder(sparse_output=False)
y_train = np.array(y_train).reshape(-1, 1)
y_train = encoder.fit_transform(y_train)

y_test = np.array(y_test).reshape(-1,1)
y_test = encoder.fit_transform(y_test)

In [ ]:
X_train = X_train.reshape(-1,1)
X_test = X_test.reshape(-1,1)

In [ ]:
X_train_reshaped = []
X_test_reshaped = []

for x in X_train:
    X_train_reshaped.append(np.array(x)[0])

for x in X_test:
    X_test_reshaped.append(np.array(x)[0])
    

In [ ]:

# X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
# y_test = np.reshape(y_test, (y_test.shape[0], y_test.shape[1],1))

Because the pixels are from 0 to 255, We have to normalize the pixels

In [ ]:
X_train_reshaped = np.array(X_train_reshaped)/255
X_test_reshaped= np.array(X_test_reshaped)/255

In [ ]:
X_train_reshaped.shape,X_test_reshaped.shape

In [ ]:
y_train.shape, y_test.shape

In [ ]:
model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),strides=(1,1),padding="valid", activation="relu", input_shape=(225,225,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3), activation="relu"))

model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())

model.add(Dense(128, activation="relu"))
model.add(Dense(2,activation="softmax"))

In [ ]:
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

In [ ]:
model.fit(X_train_reshaped, y_train, batch_size=14, epochs=20, verbose=1, validation_data=(X_test_reshaped, y_test))

In [ ]:
model.summary()

In [ ]:
pred = model.predict(X_test_reshaped[:10])

In [ ]:
np.argmax(pred, axis=1)

In [ ]:
np.argmax(y_test[:10], axis=1)